# L0

In [1]:
def lgda_solver(
    x0: np.ndarray, y0: np.ndarray,
    D: int, J: int, num_rows_columns: int,
    p: int, r: int, 
    alpha: float, beta: float, 
    h: np.ndarray, J_L: set, J_F: set,
    eta_x: float = 0.01, eta_y: float = 0.01,
    mu: float = 0.01,
    max_iter: int = 500, tau_interval: int = 10,
    tol: float = 1e-6,
    return_history: bool = False
):
    """LGDA with Mutation (Algorithm 4) solver (final one-shot projection)."""
    demand_points, candidate_sites = generate_instance(num_rows_columns, D, J, seed=42)
    distances = compute_distances(demand_points, candidate_sites)
    w = compute_wij_matrix(distances, alpha, beta)
    Ui_L = compute_Ui_L(w, J_L)
    Ui_F = compute_Ui_F(w, J_F)
    
    x = x0.copy()
    y = y0.copy()
    prev_obj = None

    cp = x.copy()
    cq = y.copy()
    tau = 0

    obj_vals, dx_vals, dy_vals = [], [], []

    for _ in range(max_iter):
        gx = grad_x(x, y, w, Ui_L, Ui_F, h)
        gy = grad_y(x, y, w, Ui_L, Ui_F, h)

        # Mutation-added update steps (no projection here)
        x_tmp = x - eta_x * gx - mu * (x - cp)
        y_tmp = y + eta_y * gy + mu * (cq - y)

        x_next = np.clip(x_tmp, 0.0, 1.0)
        y_next = np.clip(y_tmp, 0.0, 1.0)

        # print("Ui_L min/max:", np.min(Ui_L), np.max(Ui_L))
        # print("w  min/max:", np.min(w), np.max(w))
        # print("x min/max:", np.min(x_next), np.max(x_next))
        # print("y min/max:", np.min(y_next), np.max(y_next))

        dx = np.linalg.norm(x_next - x)
        dy = np.linalg.norm(y_next - y)
        new_obj = compute_Lhat(x_next, y_next, w, Ui_L, Ui_F, h)

        obj_vals.append(new_obj)
        dx_vals.append(dx)
        dy_vals.append(dy)

        # 収束判定（tol 使用）
        small_update = max(dx, dy) < tol
        small_obj_change = (prev_obj is not None) and (abs(new_obj - prev_obj) < tol)

        x, y = x_next, y_next
        prev_obj = new_obj

        # if small_update or small_obj_change:
        #     break

        tau += 1
        if tau == tau_interval:
            cp = x.copy()
            cq = y.copy()
            tau = 0

    # ---- 最後の最後に一度だけ射影 ----
    x_final = project_cardinality(x, p)
    y_final = project_cardinality(y, r, mask=(x_final > 0))
    obj_final = compute_Lhat(x_final, y_final, w, Ui_L, Ui_F, h)

    history = {
        "objective": np.array(obj_vals),
        "dx": np.array(dx_vals),
        "dy": np.array(dy_vals)
    }

    if return_history:
        return x_final, y_final, obj_final, candidate_sites, demand_points, history
    return x_final, y_final, obj_final, candidate_sites, demand_points



NameError: name 'np' is not defined

# L1

In [ ]:
def generate_instance(num_rows_columns, I, J, seed=42):
    """
    Generate I demand points and J candidate facility sites from a grid of size num_rows_columns x num_rows_columns.
    Returns: (demand_points, candidate_sites)
    """
    if seed is not None:
        random.seed(seed)
    
    # すべての格子点を生成（1始まり）
    all_points = [(x, y) for x in range(1, num_rows_columns + 1)
                         for y in range(1, num_rows_columns + 1)]
    
    # ランダムにシャッフル
    random.shuffle(all_points)
    
    # 十分な点があるか確認
    assert I + J <= len(all_points), "Grid is too small for given I and J."
    
    demand_points = all_points[:I]
    candidate_sites = all_points[I:I+J]
    
    return demand_points, candidate_sites


In [ ]:
def compute_Ui_L(wij_matrix, J_L):
    """
    リーダーの既存施設による U_i^L を計算する関数

    Parameters:
        wij_matrix (np.array): D × J の w_ij の重み行列
        J_L (set): リーダーが既に持っている施設のインデックス集合

    Returns:
        np.array: 各需要点 i に対する U_i^L のベクトル
    """
    D, _ = wij_matrix.shape  # D: 需要点の数, J: 候補施設の数

    # J_L が空なら影響はゼロ
    if not J_L:
        return np.zeros(D)

    # 各需要点 i に対して、リーダーの施設 j ∈ J_L からの重みを合計する
    utility_vector = np.zeros(D)
    for j in J_L:
        # 列 j は、施設 j が各需要点に与える重み
        utility_vector += wij_matrix[:, j]

    return utility_vector

In [ ]:
def compute_Ui_F(wij_matrix, J_F):
    """
    リーダーの既存施設による U_i^L を計算する関数

    Parameters:
        wij_matrix (np.array): D × J の w_ij の重み行列
        J_L (set): リーダーが既に持っている施設のインデックス集合

    Returns:
        np.array: 各需要点 i に対する U_i^L のベクトル
    """
    D, _ = wij_matrix.shape  # D: 需要点の数, J: 候補施設の数

    # J_L が空なら影響はゼロ
    if not J_F:
        return np.zeros(D)

    # 各需要点 i に対して、リーダーの施設 j ∈ J_L からの重みを合計する
    utility_vector = np.zeros(D)
    for j in J_F:
        # 列 j は、施設 j が各需要点に与える重み
        utility_vector += wij_matrix[:, j]

    return utility_vector

In [ ]:
def compute_wij_matrix(distances, alpha=0, beta=0.1):
    wij_matrix = np.exp(alpha - beta * distances)
    return wij_matrix

In [ ]:
def compute_distances(demand_points, candidate_sites):
    D, J = len(demand_points), len(candidate_sites)  # ここで D, J を定義
    distances = np.zeros((D, J))
    for d in range(D):
        for j in range(J):
            distances[d, j] = np.sqrt(
                (demand_points[d][0] - candidate_sites[j][0]) ** 2
                + (demand_points[d][1] - candidate_sites[j][1]) ** 2
            )
    return distances

In [ ]:
def grad_x(x: np.ndarray, y: np.ndarray, w: np.ndarray,
           Ui_L: np.ndarray, Ui_F: np.ndarray, h: np.ndarray) -> np.ndarray:
    """Gradient of \hat{L} with respect to x (ascent direction)."""
    Ai = compute_Ai(x, y, w, Ui_L)
    Bi = compute_Bi(x, y, w, Ui_L, Ui_F)

    dA_dx = w * ((1.0 + y) - 2.0 * y * x)     # shape (I, J)
    dB_dx = w * (1.0 - y)

    frac = (Bi[:, None] * dA_dx - Ai[:, None] * dB_dx) / (Bi[:, None] ** 2)
    return (frac * h[:, None]).sum(axis=0)

In [ ]:
def grad_y(x: np.ndarray, y: np.ndarray, w: np.ndarray,
           Ui_L: np.ndarray, Ui_F: np.ndarray, h: np.ndarray) -> np.ndarray:
    """Gradient of \hat{L} with respect to y (descent direction)."""
    Ai = compute_Ai(x, y, w, Ui_L)
    Bi = compute_Bi(x, y, w, Ui_L, Ui_F)

    dA_dy = w * (-x ** 2 + x)
    dB_dy = w * (1.0 - x)

    frac = (Bi[:, None] * dA_dy - Ai[:, None] * dB_dy) / (Bi[:, None] ** 2)
    return (frac * h[:, None]).sum(axis=0)

In [ ]:
def compute_Lhat(x: np.ndarray, y: np.ndarray, w: np.ndarray,
                 Ui_L: np.ndarray, Ui_F: np.ndarray, h: np.ndarray) -> float:
    """Evaluate the objective \hat{L}(x, y)."""
    Ai = compute_Ai(x, y, w, Ui_L)
    Bi = compute_Bi(x, y, w, Ui_L, Ui_F)
    return float(np.dot(h, Ai / Bi))


In [ ]:
def project_cardinality(v: np.ndarray, k: int, mask: np.ndarray | None = None) -> np.ndarray:
    """Project a vector onto the set {0,1}^J with at most *k* ones."""
    v = np.clip(v, 0.0, 1.0)
    if mask is not None:
        v = v * (~mask)

    if k >= v.size:
        return (v > 0).astype(float)

    idx = np.argpartition(-v, k)[:k]
    out = np.zeros_like(v)
    out[idx] = 1.0
    return out

# L2

In [ ]:
def compute_Ai(x: np.ndarray, y: np.ndarray, w: np.ndarray, Ui_L: np.ndarray) -> np.ndarray:
    """Compute A_i = U_i^L + Σ_j w_ij[-y_j x_j^2 + (1+y_j)x_j] for all i."""
    term = w * ((1.0 + y) * x - y * x ** 2)  # broadcast over j
    Ai = Ui_L + term.sum(axis=1)
    # print("Ai min:", np.min(Ai))
    return Ai

In [ ]:
def compute_Bi(x: np.ndarray, y: np.ndarray, w: np.ndarray,
               Ui_L: np.ndarray, Ui_F: np.ndarray) -> np.ndarray:
    """Compute B_i = U_i^L + U_i^F + Σ_j w_ij[(1-y_j)x_j + y_j] for all i."""
    term = w * ((1.0 - y) * x + y)
    Bi = Ui_L + Ui_F + term.sum(axis=1)
    # print("Bi min:", np.min(Bi))
    return Bi

# Prot

In [ ]:
def plot_minmax_history(L_vals: list | np.ndarray,
                        dx_vals: list | np.ndarray,
                        dy_vals: list | np.ndarray,
                        *, logy: bool = True):
    """Plot convergence history returned by *minmax_solver*.

    Parameters
    ----------
    L_vals : sequence of float
        Objective values (hist_Lcont).
    dx_vals : sequence of float
        Norm of x updates.
    dy_vals : sequence of float
        Norm of y updates.
    logy : bool, default True
        Use log scale for dx/dy curves.
    """
    L_vals = np.asarray(L_vals)
    dx_vals = np.asarray(dx_vals)
    dy_vals = np.asarray(dy_vals)
    iters = np.arange(1, len(L_vals) + 1)

    fig, ax1 = plt.subplots(figsize=(6, 4))
    ax1.plot(iters, L_vals, label="objective", linewidth=1.5, color="tab:blue")
    ax1.set_xlabel("iteration")
    ax1.set_ylabel("objective")

    ax2 = ax1.twinx()
    ax2.plot(iters, dx_vals, linestyle="--", label="‖dx‖", color="tab:orange")
    ax2.plot(iters, dy_vals, linestyle=":", label="‖dy‖", color="tab:green")
    ax2.set_ylabel("step size")
    if logy:
        ax2.set_yscale("log")

    lines, labels = ax1.get_legend_handles_labels()
    l2, lab2 = ax2.get_legend_handles_labels()
    ax1.legend(lines + l2, labels + lab2, loc="best")
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_facility_selection(candidate_sites, demand_points, x_bin, y_bin):
    """
    施設配置の可視化関数。
    
    Parameters
    ----------
    candidate_sites : list of tuple(float, float)
        候補施設の座標 [(x1, y1), (x2, y2), ...]
    demand_points : list of tuple(float, float)
        需要点の座標 [(x1, y1), (x2, y2), ...]
    x_bin : array-like of 0/1
        リーダーによって選ばれた施設（青丸で表示）
    y_bin : array-like of 0/1
        フォロワーによって選ばれた施設（赤丸で表示）
    """
    # 座標分解
    candidate_x = [pt[0] for pt in candidate_sites]
    candidate_y = [pt[1] for pt in candidate_sites]
    demand_x = [pt[0] for pt in demand_points]
    demand_y = [pt[1] for pt in demand_points]

    # プロット開始
    plt.figure(figsize=(8, 8))

    # 需要点（黒）
    plt.scatter(demand_x, demand_y, color='black', marker='x', label='Demand Points')

    # 候補地（グレー）
    plt.scatter(candidate_x, candidate_y, color='gray', label='Candidate Sites')

    # x_bin == 1 → 青い○（枠のみ）
    for i, val in enumerate(x_bin):
        if val == 1:
            plt.scatter(candidate_sites[i][0], candidate_sites[i][1],
                        s=200, facecolors='none', edgecolors='blue', linewidths=2,
                        label='x_bin = 1' if 'x_bin = 1' not in plt.gca().get_legend_handles_labels()[1] else "")

    # y_bin == 1 → 赤い●
    for i, val in enumerate(y_bin):
        if val == 1:
            plt.scatter(candidate_sites[i][0], candidate_sites[i][1],
                        s=100, color='red',
                        label='y_bin = 1' if 'y_bin = 1' not in plt.gca().get_legend_handles_labels()[1] else "")

    # 軸・凡例など
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Demand Points and Candidate Sites with Selections')
    plt.grid(True)
    plt.legend()
    plt.axis('equal')
    plt.show()